In [1]:
import json
import uuid

import boto3

In [9]:
class KinesisStream(object):
    
    def __init__(self, stream):
        self.stream = stream

    # internal connection method 
    # underscore means only other internal methods should use this method, not the user.
    def _connected_client(self):
        """ Connect to Kinesis Streams """
        return boto3.client('kinesis')

    def send_stream(self, data, partition_key=None):
        """
        data: python dict containing your data.
        partition_key:  set it to some fixed value if you want processing order
                        to be preserved when writing successive records.
                        
                        If your kinesis stream has multiple shards, AWS hashes your
                        partition key to decide which shard to send this record to.
                        
                        Ignore if you don't care for processing order
                        or if this stream only has 1 shard.
                        
                        If your kinesis stream is small, it probably only has 1 shard anyway.
        """

        # If no partition key is given, assume random sharding for even shard write load
        if partition_key == None:
            partition_key = str(uuid.uuid4())

        # this doesnt actually call or trigger anything - it just collects some config stuff to make your requests like
        # credentials, keys, aws region etc.
        client = self._connected_client()
        
        return client.put_record(
            StreamName=self.stream,
            Data=json.dumps(data),
            PartitionKey=partition_key
        )

In [17]:
jacobs_stream = KinesisStream('jacobs-test-stream')
jacobs_stream.send_stream(data = {'my': 'data'})

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the PutRecord operation: Stream jacobs-test-stream under account 324816727452 not found.